In [0]:
#downloading the dataset
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [0]:
!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

In [0]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [0]:
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [0]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [0]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [0]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [0]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [0]:
model_vgg = Sequential()

In [0]:
model_vgg.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

58892288/58889256 [==============================] - 1s 0us/step


In [0]:
model_vgg.add(Dense(num_classes, activation='softmax'))

In [0]:
model_vgg.layers

In [0]:
model_vgg.layers[0].trainable = False

In [0]:
model_vgg.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [0]:
model_vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [0]:
fit_history = model_vgg.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1
)

Epoch 1/2
301/301 [==============================] - 136s 453ms/step - loss: 0.1093 - accuracy: 0.9656 - val_loss: 0.0012 - val_accuracy: 0.9941
Epoch 2/2
301/301 [==============================] - 132s 438ms/step - loss: 0.0229 - accuracy: 0.9948 - val_loss: 0.0099 - val_accuracy: 0.9954


In [0]:
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle=False)

Found 500 images belonging to 2 classes.


In [0]:
scores_vgg = model_vgg.evaluate_generator(test_generator)

In [0]:
print('VGG16 model test accuracy:',scores_vgg[1])
print('VGG16 model test loss:',scores_vgg[0])

VGG16 model test accuracy: 0.9959999918937683
VGG16 model test loss: 0.0019834833219647408


In [0]:
pred = model_vgg.predict_generator(test_generator)

In [0]:
import numpy as np
preds = np.argmax(pred, axis=1)

In [0]:
model_vgg.save('classifier_vgg16_model.h5')